In [7]:
import pandas as pd


# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [13]:
data = pd.read_csv('../data/adult.csv')
data.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [28]:
df = data[~data.isin(['?']).any(axis=1)]


,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
count,30162.000000,30162.000000,30162,30162,30162,30162,30162,30162,30162,30162.000000,30162,30162,30162.000000
unique,NaN,NaN,7,16,7,14,6,5,2,NaN,41,2,NaN
top,NaN,NaN,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,United-States,<=50K,NaN
freq,NaN,NaN,22286,9840,14065,4038,12463,25933,20380,NaN,27504,22654,NaN
mean,16287.103242,38.437902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.931238,NaN,NaN,73.968570
std,9383.401676,13.134665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.979984,NaN,NaN,85.365144
min,0.000000,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,15.000000
25%,8178.250000,28.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,26.000000
50%,16285.500000,37.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.000000,NaN,NaN,38.000000
75%,24401.750000,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.000000,NaN,NaN,49.000000


# Task 1
Print the count of men and women in the dataset.

In [34]:
men = df[df['sex'] == "Male"].value_counts().count()
women = df[df['sex'] == "Female"].value_counts().count()
print(men, women)

20380 9782


# Task 2
Find the average age of men in dataset

In [38]:
df[df['sex'] == 'Male']['age'].mean()


np.float64(39.18400392541707)

# Task 3
Get the percentage of people from Poland (native-country)

In [41]:
is_pole = df[df['native-country'] == 'Poland'].value_counts().count() / df.value_counts().sum()
print('{:,.2%}'.format(is_pole))

0.19%


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [45]:
df[df['salary K$'] > 50]['age'].mean()


np.float64(43.95911028236548)

In [46]:
df[df['salary K$'] > 50]['age'].std()

np.float64(10.269632835673852)

In [47]:
df[df['salary K$'] <= 50]['age'].mean()

np.float64(36.60806038668668)

In [48]:
df[df['salary K$'] <= 50]['age'].std()

np.float64(13.464631257161633)

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [53]:
higher_education = ['Bachelors', 'Masters', 'Assoc-acdm', 'Doctorate', 'Assoc-voc', 'Prof-school']
any(df['education'].isin(higher_education) & df['salary K$'] > 50)

False

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [65]:
df['age'].groupby(df['education']).describe()



,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [69]:
married_men_salary_ave = df[df['marital-status'].str.startswith('Married')]['salary K$'].mean()
print(married_men_salary_ave)

107.3532097399004


In [73]:
not_married_men_salary_ave = df[~df['marital-status'].str.startswith('Married')]['salary K$'].mean()
print(not_married_men_salary_ave)

43.240927034254426


In [71]:
married_men_salary_ave > not_married_men_salary_ave

np.True_

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [90]:
most_time_worked = df['hours-per-week'].max()
print(most_time_worked)

99


In [93]:
df[df['hours-per-week'] == most_time_worked]['hours-per-week'].count()


np.int64(78)

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [95]:
corr_matrix = df.corr(numeric_only=True)
print(corr_matrix)

                Unnamed: 0       age  hours-per-week  salary K$
Unnamed: 0        1.000000 -0.001126       -0.001890   0.000129
age              -0.001126  1.000000        0.101599   0.208203
hours-per-week   -0.001890  0.101599        1.000000   0.196378
salary K$         0.000129  0.208203        0.196378   1.000000
